In [2]:
from math import sqrt
from scipy import stats

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from env import get_db_url
from wrangle import wrangle_telco

As a customer analyst, I want to know who has spent the most money with us over their lifetime. I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. I need to do this within an average of $5.00 per customer.

In [ ]:
sql = """
SELECT customer_id, monthly_charges, tenure, total_charges
FROM customers
WHERE contract_type_id = 3
"""

In [ ]:
url = get_db_url('telco_churn')
tc_df = pd.read_sql(sql, url)
tc_df.total_charges = tc_df.total_charges.str.replace(' ', '0').astype(float)
tc_df

In [ ]:
# changed the function slightly in wrangle.py
#def wrangle_telco():
#    sql = """
#    SELECT customer_id, monthly_charges, tenure, total_charges
#    FROM customers
#    WHERE contract_type_id = 3
#    """
#    url = get_db_url('telco_churn')
#    tc_df = pd.read_sql(sql, url)
#    tc_df.total_charges = tc_df.total_charges.str.replace(' ', '0').astype(float)
#    return tc_df

In [4]:
tc_df = wrangle_telco()
tc_df

,monthly_charges,tenure,total_charges
customer_id,,,
0013-SMEOE,109.70,71,7904.25
0014-BMAQU,84.65,63,5377.80
0016-QLJIS,90.45,65,5957.90
0017-DINOC,45.20,54,2460.55
0017-IUDMW,116.80,72,8456.75
...,...,...,...
9964-WBQDJ,24.40,71,1725.40
9972-EWRJS,19.25,67,1372.90
9975-GPKZU,19.75,46,856.50


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
train_test_split?

In [ ]:
# 1. Make the thing
scaler = MinMaxScaler()
# 2. Fit the thing
scaler.fit(tc_df[['monthly_charges', 'tenure', 'total_charges']])
# 3. Use the thing
scaler.transform(tc_df[['monthly_charges', 'tenure', 'total_charges']])

In [10]:
X = tc_df[['monthly_charges', 'tenure']]
y = tc_df[['total_charges']]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 294)
X_train, X_test, y_train, y_test

(             monthly_charges  tenure
 customer_id                         
 3460-TJBWI             24.20      24
 8295-KMENE             76.45      59
 8039-EQPIM             60.25      69
 3777-XROBG             19.55      58
 4884-TVUQF            101.30      57
 ...                      ...     ...
 0847-HGRML             20.00      62
 8532-UEFWH             25.75      52
 5879-HMFFH             88.05      72
 4853-RULSV            104.00      70
 8857-CUPFQ             19.25      63
 
 [1356 rows x 2 columns],              monthly_charges  tenure
 customer_id                         
 6285-FTQBF             25.55      72
 8871-JLMHM             90.20      68
 8311-UEUAB             75.50      67
 5248-KWLAR             90.35      71
 4139-DETXS             64.45      72
 ...                      ...     ...
 9135-MGVPY             84.70      69
 2300-RQGOI             20.05      38
 8237-ULIXL             24.85      72
 5310-NOOVA             19.95      46
 7033-CLAMM            

In [ ]:
def split_my_data(X, y, train_pct):
    return train_test_split(X, y, train_size = train_pct, random_state = 294)

In [15]:
std_scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X_train)
train_scaled_std = pd.DataFrame(std_scaler.transform(X_train), columns=X_train.columns.values).set_index([X_train.index.values])
test_scaled_std = pd.DataFrame(std_scaler.transform(X_test), columns=X_test.columns.values).set_index([X_test.index.values])
train_scaled_std

,monthly_charges,tenure
3460-TJBWI,-1.046584,-1.809541
8295-KMENE,0.468623,0.118199
8039-EQPIM,-0.001163,0.668982
3777-XROBG,-1.181430,0.063121
4884-TVUQF,1.189253,0.008042
...,...,...
0847-HGRML,-1.168380,0.283434
8532-UEFWH,-1.001635,-0.267349
5879-HMFFH,0.805014,0.834217
4853-RULSV,1.267551,0.724060


In [16]:
test_scaled_std

,monthly_charges,tenure
6285-FTQBF,-1.007435,0.834217
8871-JLMHM,0.867362,0.613904
8311-UEUAB,0.441074,0.558825
5248-KWLAR,0.871712,0.779138
4139-DETXS,0.120633,0.834217
...,...,...
9135-MGVPY,0.707867,0.668982
2300-RQGOI,-1.166930,-1.038445
8237-ULIXL,-1.027734,0.834217
5310-NOOVA,-1.169830,-0.597819


In [ ]:
def standard_scaler(test, train):
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(train)
    train_scaled_std = pd.DataFrame(scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled_std = pd.DataFrame(scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return std_scaler, train_scaled_std, test_scaled_std

In [ ]:
def scale_inverse():
    train_unscaled = pd.DataFrame(std_scaler.inverse_transform(train_scaled_std), columns=train_scaled_std.columns.values).set_index([train_scaled_std.index.values])
    test_unscaled = pd.DataFrame(std_scaler.inverse_transform(test_scaled_std), columns=test_scaled_std.columns.values).set_index([test_scaled_std.index.values])
    return train_unscaled, test_unscaled

In [ ]:
def uniform_scaler(train, test):
    unf_scaler = QuantileTransformer(n_quantiles=100, output_distribution='uniform', random_state=123, copy=True).fit(train)
    train_scaled_unf = pd.DataFrame(unf_scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled_unf = pd.DataFrame(unf_scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return unf_scaler, train_scaled_unf, test_scaled_unf

In [ ]:
def gaussian_scaler(train, test):
    gs_scaler = PowerTransformer(method='yeo-johnson', standardize=False, copy=True).fit(train)
    train_scaled_gs = pd.DataFrame(gs_scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled_gs = pd.DataFrame(gs_scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return gs_scaler, train_scaled_gs, test_scaled_gs

In [ ]:
def min_max_scaler(train, test):
    mm_scaler = MinMaxScaler(copy=True, feature_range=(0,1)).fit(train)
    train_scaled_mm = pd.DataFrame(mm_scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled_mm = pd.DataFrame(mm_scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return mm_scaler, train_scaled_mm, test_scaled_mm

In [ ]:
def iqr_robust_scaler(train, test):
    iqr_scaler = RobustScaler(quantile_range=(25.0,75.0), copy=True, with_centering=True, with_scaling=True).fit(train)
    train_scaled_rob = pd.DataFrame(iqr_scaler.transform(train), columns=train.columns.values).set_index([train.index.values])
    test_scaled_rob = pd.DataFrame(iqr_scaler.transform(test), columns=test.columns.values).set_index([test.index.values])
    return train_scaled_rob, test_scaled_rob